In [ ]:
import cv2
import time
import torch
from facenet_pytorch import MTCNN


In [ ]:

def run_eye_detection_only(desired_fps=10, use_camera_index=0):
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    mtcnn = MTCNN(keep_all=True, device=device)

    video_capture = cv2.VideoCapture(use_camera_index)
    frame_delay = 1 / desired_fps

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        boxes, probs, landmarks = mtcnn.detect(rgb_frame, landmarks=True)

        eyes_detected = False

        if landmarks is not None:
            for landmark_set in landmarks:
                left_eye = landmark_set[0]
                right_eye = landmark_set[1]

                if left_eye is not None and right_eye is not None:
                    eyes_detected = True
                    # Draw circles on the eyes
                    cv2.circle(frame, (int(left_eye[0]), int(left_eye[1])), 3, (255, 0, 0), -1)
                    cv2.circle(frame, (int(right_eye[0]), int(right_eye[1])), 3, (255, 0, 0), -1)

        print("Eyes detected" if eyes_detected else "Eyes not detected")

        cv2.imshow('Eye Detection (MTCNN)', frame)

        time.sleep(frame_delay)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()


In [15]:
if __name__ == "__main__":
    run_eye_detection_only()


Eyes not detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
Eyes detected
